Importo las librerías que necesito.

In [162]:
import pandas as pd
import numpy as np 
import ast
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors

Traigo los DataFrame creados durante el ETL.

In [83]:
ruta_games = os.path.join(r'D:\1er Proyecto\datasets_finales\games.parquet')
df_games = pd.read_parquet(ruta_games)

In [84]:
ruta_review = os.path.join(r'D:\1er Proyecto\datasets_finales\reviews.parquet')
df_review = pd.read_parquet(ruta_review)

In [85]:
ruta_item = os.path.join(r'D:\1er Proyecto\datasets_finales\user_items.parquet')
df_item = pd.read_parquet(ruta_item)

In [178]:
def developer(empresa: str, df=df_games):
    # Convertir el nombre de la empresa a minúsculas para hacer la búsqueda insensible a mayúsculas y minúsculas
    empresa = empresa.lower()
    
    # Filtrar el DataFrame por la empresa desarrolladora que contiene el nombre especificado
    df_empresa = df[df['developer'].str.lower().str.contains(empresa)]
    
    # Verificar si hay datos para la empresa desarrolladora especificada
    if df_empresa.empty:
        print(f"No hay datos disponibles para la empresa desarrolladora que contiene '{empresa}'.")
        return None
        
    # Agrupar por año
    grouped = df_empresa.groupby('year')
    
    # Calcular cantidad de items y porcentaje de contenido Free por año
    resultados = []
    for year, group in grouped:
        total_items = len(group)
        free_items = len(group[group['price'] == 0])
        porcentaje_free = (free_items / total_items) * 100 if total_items > 0 else 0
        resultados.append({
            'Año': year,
            'Cantidad de Items': total_items,
            'Porcentaje Free': porcentaje_free
        })
    
    # Ordenar los resultados por año
    resultados.sort(key=lambda x: x['Año'])
    
    return resultados

In [179]:
#Ejemplo
developer('valve')

[{'Año': 1998, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 1999, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 2000, 'Cantidad de Items': 2, 'Porcentaje Free': 0.0},
 {'Año': 2001, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 2003, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 2004, 'Cantidad de Items': 5, 'Porcentaje Free': 0.0},
 {'Año': 2005, 'Cantidad de Items': 1, 'Porcentaje Free': 100.0},
 {'Año': 2006, 'Cantidad de Items': 2, 'Porcentaje Free': 0.0},
 {'Año': 2007, 'Cantidad de Items': 3, 'Porcentaje Free': 33.33333333333333},
 {'Año': 2008, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 2009, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 2010, 'Cantidad de Items': 2, 'Porcentaje Free': 50.0},
 {'Año': 2011, 'Cantidad de Items': 1, 'Porcentaje Free': 0.0},
 {'Año': 2012, 'Cantidad de Items': 2, 'Porcentaje Free': 50.0},
 {'Año': 2014, 'Cantidad de Items': 8, 'Porcentaje Free': 87.5},
 {'Año': 2015, 'Canti

In [94]:
def userdata(user_name, df_games=df_games, df_items=df_item, df_reviews=df_review) -> dict:
    #Identificación usuario
    user_name = str(user_name)
    #Obtener cantidad de items
    user_items = df_items[df_items['user_id'] == user_name] #conexión de items de un usuario por su id
    num_items = len(user_items['item_id'].unique())
    #Obtención de dinero gastado
    user_items_prices = user_items.merge(df_games, left_on='item_id', right_on='item_id', how='inner')
    total_gastado = user_items_prices['price'].sum()
    #Porcentaje de recomendación
    user_reviews = df_reviews[df_reviews['user_id'] == user_name] #conexión de reviews de un usuario por su id
    total_reviews = len(user_reviews)
    positive_reviews = user_reviews[user_reviews['recommend']==1]
    num_positive_reviews = len(positive_reviews)
    if total_reviews != 0:
        porcentaje_positive_reviews = (num_positive_reviews / total_reviews) * 100
    else:
        porcentaje_positive_reviews = 0
    # Crear diccionario con la información
    user_data = {
        'Total gastado (USD)': total_gastado,
        'Porcentaje de recomendación positiva': porcentaje_positive_reviews,
        'Cantidad de juegos del usuario': num_items
    }
    return user_data

In [100]:
#Ejemplo
userdata('imsodonionringsrightnow')

{'Total gastado (USD)': 360.58000000000004,
 'Porcentaje de recomendación positiva': 77.77777777777779,
 'Cantidad de juegos del usuario': 51}

In [121]:
#Creo un DataFrame de géneros
df_genres = df_games.copy()
df_genres['genres'] = df_genres['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
df_genres.drop(['name', 'tags','specs','price','developer'], axis=1)

,genres,item_id,year
0,[[Action]],10,2000
1,[[Indie]],1002,2005
2,"[[Animation &amp, Modeling]]",100400,2012
3,[[Action]],10090,2008
4,"[[Animation &amp, Modeling]]",100980,2012
...,...,...,...
23940,[[Action]],9970,2011
23941,"[[Casual, Indie, Strategy]]",99700,2011
23942,[[Casual]],9980,2010
23943,"[[Adventure, Casual, Free to Play, Massivel...",99910,2011


In [128]:
def UserForGenre(genero: str):
  
  genero = '[' + genero + ']'
  
  # Filtrar el DataFrame para dejar solo los juegos que contengan el género especificado
  df_genre = df_genres[df_genres['genres'].apply(lambda x: genero in x if isinstance(x, list) else False)]

  # Filtrar los usuarios que poseen los juegos del género específico
  df_user_aggregated = df_item[df_item['item_id'].isin(df_genre['item_id'])]
    
  # Merge para concatenar el año de df_genre a df_user_aggregated basado en el item_id
  df_user_aggregated = df_user_aggregated.merge(df_genre[['item_id', 'year']], on='item_id', how='left')
    
  # Calcular la suma de las horas jugadas por cada usuario a los juegos del género específico
  user_hours_per_game = df_user_aggregated.groupby('user_id')['playtime_forever'].sum()
    
  # Obtener al usuario con más horas jugadas
  user_most_hours_user_id = user_hours_per_game.idxmax()
    
  # Filtrar las horas jugadas por el usuario con más horas jugadas
  user_most_hours_df = df_user_aggregated[df_user_aggregated['user_id'] == user_most_hours_user_id]
    
  # Calcular la cantidad de horas jugadas por año del usuario con más horas jugadas considerando el año de publicación del juego
  hours_per_year = user_most_hours_df.groupby('year')['playtime_forever'].sum().reset_index()
    
  # Formatear el resultado en el formato especificado
  result = {
        "Usuario con más horas jugadas para " + genero: user_most_hours_user_id,
        "Horas jugadas": [{"Año": int(row['year']), "Horas": int(row['playtime_forever'])} for index, row in hours_per_year.iterrows()]
  }
    
  return result

In [135]:
#Ejemplo
UserForGenre('Action')

{'Usuario con más horas jugadas para [Action]': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1999, 'Horas': 44},
  {'Año': 2000, 'Horas': 70644},
  {'Año': 2001, 'Horas': 13},
  {'Año': 2002, 'Horas': 238},
  {'Año': 2003, 'Horas': 7673},
  {'Año': 2004, 'Horas': 127411},
  {'Año': 2005, 'Horas': 21339},
  {'Año': 2006, 'Horas': 652},
  {'Año': 2007, 'Horas': 109611},
  {'Año': 2009, 'Horas': 103589},
  {'Año': 2010, 'Horas': 48437},
  {'Año': 2011, 'Horas': 15092},
  {'Año': 2012, 'Horas': 266977},
  {'Año': 2013, 'Horas': 4860},
  {'Año': 2014, 'Horas': 20717},
  {'Año': 2015, 'Horas': 63380}]}

In [180]:
def best_developer_year(año: int, games_df=df_games, reviews_df=df_review):
    games_df.sort_values(by='item_id', ascending=False, inplace=True, ignore_index=True)
    reviews_df.sort_values(by='item_id', ascending=False, inplace=True, ignore_index=True)
    # Filtrar juegos por el año especificado
    games_filtered = games_df[games_df['year'] == año]
    # Merge juegos y reviews basado en 'id'
    merged_df = pd.merge(games_filtered, reviews_df, left_on='item_id', right_on='item_id')
    # Contar recomendaciones por desarrollador
    developer_counts = merged_df[merged_df['recommend'] == 1]['developer'].value_counts().reset_index()
    developer_counts.columns = ['Developer', 'Recommendations']
    # Ordenar por número de recomendaciones y obtener los 3 primeros
    sorted_developers = developer_counts.sort_values(by='Recommendations', ascending=False)
    top_developers = sorted_developers.head(3)
    # Formatear el resultado como una lista de diccionarios
    result = [{"Puesto {}: {}".format(i+1, row['Developer']): row['Recommendations']} for i, row in top_developers.iterrows()]
    return result

In [181]:
#Ejemplo
best_developer_year(2015)

[{'Puesto 1:  Psyonix, Inc.': 207},
 {'Puesto 2:  Rockstar North': 188},
 {'Puesto 3:  tobyfox': 178}]

In [140]:
def developer_reviews_analysis(desarrolladora: str, games_df=df_games, reviews_df=df_review):
    games_df.sort_values(by='item_id', ascending=False, inplace=True, ignore_index=True)
    reviews_df.sort_values(by='item_id', ascending=False, inplace=True, ignore_index=True)
    # Merge juegos y reviews basado en 'id'
    merged_df = pd.merge(games_df, reviews_df, left_on='item_id', right_on='item_id')
    # Filtrar por la desarrolladora especificada
    developer_filtered = merged_df[merged_df['developer'].str.strip().str.lower() == desarrolladora.lower()]
    # Contar registros de reseñas categorizadas como análisis positivo o negativo
    positive_count = len(developer_filtered[developer_filtered['sentiment_analysis'] == 2])
    negative_count = len(developer_filtered[developer_filtered['sentiment_analysis'] == 0])
    # Crear el diccionario de retorno
    result = {desarrolladora: {'Negative': negative_count, 'Positive': positive_count}}
    return result

In [141]:
developer_reviews_analysis('Valve')

{'Valve': {'Negative': 942, 'Positive': 4820}}

Función recomendación de juegos

El modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello aplicaré la similitud del coseno.

In [168]:
def recomendacion_juego(product_id: int):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_games['genres'])
    
    # Configurar el modelo "Vecino más cercano"
    nn = NearestNeighbors(metric='cosine', algorithm='brute')
    nn.fit(tfidf_matrix)
    
    # Obtener el índice del juego actual
    juego_index = df_games[df_games['item_id'] == product_id].index[0]
    
    # Encontrar los índices de los 5 juegos más similares
    distances, indices = nn.kneighbors(tfidf_matrix[juego_index], n_neighbors=6)
    
    # Excluir el propio juego de los resultados
    similar_indices = indices.flatten()[1:]
    
    # Obtener los nombres de los juegos similares
    juegos_similares = df_games.iloc[similar_indices]['name'].values
    
    # Obtener el nombre del juego actual
    juego_actual = df_games.loc[df_games['item_id'] == product_id, 'name'].iloc[0]
    
    # Crear el resultado en el formato especificado
    resultado = {
        "Juego actual": juego_actual,
        "Juegos recomendados similares": list(juegos_similares)
    }
    
    return resultado

In [177]:
recomendacion_juego(2028056)

{'Juego actual': ' Worms Revolution Season Pass',
 'Juegos recomendados similares': [' Elven Legacy: Ranger',
  ' Worms Reloaded: Time Attack Pack',
  ' Worms Reloaded: Forts Pack',
  ' Elven Legacy: Siege',
  ' Worms Reloaded: Puzzle Pack']}